In [2]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pickle

from pathlib import Path
from itertools import product

from pipeline_vsdi import cleaning as cl
from pipeline_vsdi.embedding import autoencoder as ae

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split


In [3]:
# PATHS AND PARAMETERS
DATA_PATH = Path('/Volumes/imaging1/davide/ATC_Data_preprocessed')
OUTPUT_BASE_PATH = Path('/Users/davide/Dropbox/Projects/ATC/autoencoders')

MODEL_PATH = OUTPUT_BASE_PATH.joinpath('models')

MODEL_PATH.mkdir(parents=True, exist_ok=True)

save_model = True
save_history = True

# Initialize variables
animals = ['A04', 'A06', 'A07', 'A08']
days = ['Day1','Day3','Day5','Day7']


BATCH_SIZE = 256
N_EPOCHS = 1
LEARNING_RATE = pow(10,-4)

LATENT_DIM = 10


criterion = nn.MSELoss()


In [6]:
for animal,day in product(animals,days):

    print(f'TRAINING AUTOENCODER FOR ANIMAL: {animal}, DAY {day}')


    # load data
    vsdi1 = loadmat(DATA_PATH.joinpath(f'{animal}/{day}/vsdi_ATC1.mat'))['vsdi_data']
    vsdi2 = loadmat(DATA_PATH.joinpath(f'{animal}/{day}/vsdi_ATC2.mat'))['vsdi_data']
    mask = loadmat(DATA_PATH.joinpath(f'{animal}/{day}/vsdi_mask.mat'))['mask']

    # prepare data
    X = np.concatenate([vsdi1,vsdi2],axis=-1)
    
    # clear memory
    del vsdi1
    del vsdi2

    # concatenate sessions

    print('Preporcessing data ... ')

    X = cl.clean_outliers(X)
    # reshape in (n_sapmples,n_channels,n_pixels,n_pixels) format
    X = np.transpose(X,(2,0,1))
    X = X[:,np.newaxis,:,:]
    # scale each imahe independently betw. 0 and 1
    d = np.max(X,axis=(1,2,3))[:, np.newaxis, np.newaxis, np.newaxis] - np.min(X,axis=(1,2,3))[:, np.newaxis, np.newaxis, np.newaxis]
    X = (X - np.min(X,axis=(1,2,3))[:, np.newaxis, np.newaxis, np.newaxis]) / d

    train_size = 80*X.shape[0]//100
    valid_size = X.shape[0] - train_size
    train, valid = random_split(X,[train_size,valid_size])
    trainloader = DataLoader(train, batch_size=BATCH_SIZE)
    validloader = DataLoader(valid, batch_size=BATCH_SIZE)

    h = X.shape[-1]
    w = X.shape[-2]
    
    # model definition and compliation
    model = ae.Conv_AE(LATENT_DIM,h_pixels=h,w_pixels=w)
    optimizer = optim.Adam(model.parameters(), lr=pow(10,-4), weight_decay=0)

    print('Starting training job ...')

    history = ae.train_autoencoder(model,trainloader,
                            validloader=validloader,
                            epochs = N_EPOCHS)
    
    if save_history:
        history_path = MODEL_PATH.joinpath(f'history_{animal}_day.pickle')
        with open(history_path, "wb") as file:
            pickle.dump(history, file)


    if save_model:
        torch.save(model,MODEL_PATH.joinpath(f'animal_{animal}_day{day}'))

    
    print('Done.')

    
        


TRAINING AUTOENCODER FOR ANIMAL: A04, DAY Day1
Preporcessing data ... 
Starting training job ...


Epoch 1/1, batch loss: 0.0176: 100%|██████████| 188/188 [09:54<00:00,  3.16s/it]


Epoch 1 		 Training Loss: 0.021938336993310045 		 Validation Loss: 0.08375311531919113
Done.
TRAINING AUTOENCODER FOR ANIMAL: A04, DAY Day3
